# Welcome To My SpaceShip Titanic Ai Model

In this model, the problem will be tried to be solved with the SVM on the binary classification problem.

In [569]:
import pandas as pd
import numpy as np

In [570]:
df_train = pd.read_csv("../../database/spceship_titanic/train.csv")
df_test = pd.read_csv("../../database/spceship_titanic/test.csv")

## Data Review

In [571]:
df_train.shape

(8693, 14)

In [572]:
df_test.shape

(4277, 13)

Let's make a copy of the file to use the PassengerId property later.

In [573]:
df_test_original = df_test.copy()

Converting bool columns to numeric columns. Beware of null values.

In [574]:
df_train["Transported"] = df_train["Transported"].astype(int)

df_train["CryoSleep"] = df_train["CryoSleep"].fillna(-1).astype(int)
df_train["VIP"] = df_train["VIP"].fillna(-1).astype(int)
df_test["CryoSleep"] = df_test["CryoSleep"].fillna(-1).astype(int)
df_test["VIP"] = df_test["VIP"].fillna(-1).astype(int)

# Bu adımdan sonra -1 olanları tekrar NaN'a çevirebiliriz
df_train["CryoSleep"].replace({-1: np.nan}, inplace=True)
df_train["VIP"].replace({-1: np.nan}, inplace=True)
df_test["CryoSleep"].replace({-1: np.nan}, inplace=True)
df_test["VIP"].replace({-1: np.nan}, inplace=True)

In [575]:
df_train['Group'] = df_train['PassengerId'].apply(lambda x: x.split('_')[0])
df_train['Group'] = df_train['Group'].astype(int)


In [576]:
df_test['Group'] = df_test['PassengerId'].apply(lambda x: x.split('_')[0])
df_test['Group'] = df_test['Group'].astype(int)


In [577]:
df_train = df_train.drop(["PassengerId","Name"],axis=1)
df_test = df_test.drop(["PassengerId","Name"],axis=1)

In [578]:
len(df_train["Cabin"].unique())

6561

By expanding the cabin feature, we will only use the knowledge of which sector the passengers sit in.

In [579]:
df_train['Cabin'] = df_train['Cabin'].fillna('Unknown')
df_train['Cabin'] = df_train['Cabin'].apply(lambda x: x[0])

df_test['Cabin'] = df_test['Cabin'].fillna('Unknown')
df_test['Cabin'] = df_test['Cabin'].apply(lambda x: x[0])

In [580]:
df_train["Cabin"].unique()

array(['B', 'F', 'A', 'G', 'U', 'E', 'D', 'C', 'T'], dtype=object)

In [581]:
df_train['CryoSleep'].fillna(df_train['CryoSleep'].mode()[0], inplace=True)
df_train['Age'].fillna(df_train['Age'].mean(), inplace=True)
df_train['VIP'].fillna(df_train['VIP'].mode()[0], inplace=True)

df_test['CryoSleep'].fillna(df_train['CryoSleep'].mode()[0], inplace=True) 
df_test['Age'].fillna(df_train['Age'].mean(), inplace=True)
df_test['VIP'].fillna(df_train['VIP'].mode()[0], inplace=True) 

In [ ]:
bins = [0, 18, 35, 60, np.inf]  
labels = ['Child', 'Young_Adult', 'Adult', 'Senior']

df_train['AgeGroup'] = pd.cut(df_train['Age'], bins=bins, labels=labels)
df_train = df_train.drop('Age', axis=1)

df_test['AgeGroup'] = pd.cut(df_test['Age'], bins=bins, labels=labels)
df_test = df_test.drop('Age', axis=1)

In [ ]:
df_train.AgeGroup.value_counts()

AgeGroup
Young_Adult    4315
Adult          2293
Child          1687
Senior          220
Name: count, dtype: int64

In [ ]:
categorical_cols = [ 'AgeGroup','HomePlanet', 'Cabin', 'Destination']


In [ ]:
df_test.head()

SyntaxError: invalid syntax (2969620169.py, line 1)

In [ ]:
def encode_and_drop(df, columns_to_encode):
    for column in columns_to_encode:
        dummies = pd.get_dummies(df[column], prefix=column, dummy_na= True)
        df = pd.concat([df, dummies], axis=1)
        df = df.drop(column, axis=1)
    return df

In [ ]:
df_train = encode_and_drop(df_train, categorical_cols)
df_test = encode_and_drop(df_test, categorical_cols)

In [ ]:
df_train[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']] = df_train[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].fillna(0)
df_test[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']] = df_test[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].fillna(0)

In [ ]:
df_train['TotalExpenditure'] = df_train['RoomService'] + df_train['FoodCourt'] + df_train['ShoppingMall'] + df_train['Spa'] + df_train['VRDeck']
df_test['TotalExpenditure'] = df_test['RoomService'] + df_test['FoodCourt'] + df_test['ShoppingMall'] + df_test['Spa'] + df_test['VRDeck']

In [ ]:
df_train = df_train.drop(['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'], axis=1)
df_test = df_test.drop(['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'], axis=1)

In [ ]:
df_train['VIP'] = df_train['VIP'].astype('bool')
df_train['CryoSleep'] = df_train['CryoSleep'].astype('bool')

df_test['VIP'] = df_test['VIP'].astype('bool')
df_test['CryoSleep'] = df_test['CryoSleep'].astype('bool')

In [ ]:
df_train['VIP_Cryo'] = df_train['VIP'] & df_train['CryoSleep']
df_test['VIP_Cryo'] = df_test['VIP'] & df_test['CryoSleep']

In [ ]:
# df_train= df_train.drop(["TotalExpenditure"],axis=1)
# df_test= df_test.drop(["TotalExpenditure"],axis=1)

In [ ]:
df_train.isnull().sum()

CryoSleep                    0
VIP                          0
Transported                  0
Group                        0
NumberInGroup                0
AgeGroup_Child               0
AgeGroup_Young_Adult         0
AgeGroup_Adult               0
AgeGroup_Senior              0
AgeGroup_nan                 0
HomePlanet_Earth             0
HomePlanet_Europa            0
HomePlanet_Mars              0
HomePlanet_nan               0
Cabin_A                      0
Cabin_B                      0
Cabin_C                      0
Cabin_D                      0
Cabin_E                      0
Cabin_F                      0
Cabin_G                      0
Cabin_T                      0
Cabin_U                      0
Cabin_nan                    0
Destination_55 Cancri e      0
Destination_PSO J318.5-22    0
Destination_TRAPPIST-1e      0
Destination_nan              0
TotalExpenditure             0
VIP_Cryo                     0
dtype: int64

In [ ]:
df_test.isnull().sum()

CryoSleep                    0
VIP                          0
Group                        0
NumberInGroup                0
AgeGroup_Child               0
AgeGroup_Young_Adult         0
AgeGroup_Adult               0
AgeGroup_Senior              0
AgeGroup_nan                 0
HomePlanet_Earth             0
HomePlanet_Europa            0
HomePlanet_Mars              0
HomePlanet_nan               0
Cabin_A                      0
Cabin_B                      0
Cabin_C                      0
Cabin_D                      0
Cabin_E                      0
Cabin_F                      0
Cabin_G                      0
Cabin_T                      0
Cabin_U                      0
Cabin_nan                    0
Destination_55 Cancri e      0
Destination_PSO J318.5-22    0
Destination_TRAPPIST-1e      0
Destination_nan              0
TotalExpenditure             0
VIP_Cryo                     0
dtype: int64

In [ ]:
df_train.dtypes

CryoSleep                       bool
VIP                             bool
Transported                    int32
Group                          int32
NumberInGroup                  int32
AgeGroup_Child                  bool
AgeGroup_Young_Adult            bool
AgeGroup_Adult                  bool
AgeGroup_Senior                 bool
AgeGroup_nan                    bool
HomePlanet_Earth                bool
HomePlanet_Europa               bool
HomePlanet_Mars                 bool
HomePlanet_nan                  bool
Cabin_A                         bool
Cabin_B                         bool
Cabin_C                         bool
Cabin_D                         bool
Cabin_E                         bool
Cabin_F                         bool
Cabin_G                         bool
Cabin_T                         bool
Cabin_U                         bool
Cabin_nan                       bool
Destination_55 Cancri e         bool
Destination_PSO J318.5-22       bool
Destination_TRAPPIST-1e         bool
D

### Data scaling

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()

In [ ]:
df_train[['TotalExpenditure']] = scaler.fit_transform(df_train[['TotalExpenditure']])

In [ ]:
df_test[['TotalExpenditure']]  = scaler.transform(df_test[['TotalExpenditure']] )

In [ ]:
X_train = df_train.drop('Transported', axis=1)

In [ ]:
y_train = df_train['Transported']


In [ ]:
X_test = df_test

## Model Selection and Training

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

In [ ]:
model = SVC(kernel= 'rbf')

In [ ]:
scores = cross_val_score(model, X_train, y_train, cv = 10)

In [ ]:
print("Cross-validation scores:", scores)
print("Average cross-validation score:", scores.mean())

Cross-validation scores: [0.49655172 0.49655172 0.49655172 0.71921749 0.50287687 0.50402762
 0.50402762 0.50402762 0.55466053 0.49597238]
Average cross-validation score: 0.5274465298996073


In [ ]:
model.fit(X_train,y_train)

SVC()

In [ ]:
predicts = model .predict(X_test)

In [ ]:
output = pd.DataFrame({'PassengerId': df_test_original.PassengerId, 'Transported': predicts.astype(bool)})

In [ ]:
output.to_csv('SVC_submission.csv', index=False)